In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
from gspp.data.datamodule import PertDataModule
from omegaconf import DictConfig, OmegaConf

### generate predictions of TxPert, TxPert-LLM, data-flow

In [5]:
import wandb
dataset = 'K562_single_cell_line'

# model_variant = "txpert"
# wandb_name = '3x4k8675'
# ckpt_name = 'best-epoch=47-step=62544.ckpt'

# model_variant = "txpert-llm"
# wandb_name = '8otxvau0'
# ckpt_name = 'best-epoch=29-step=39090.ckpt'

# model_variant = "control-mean-flow"
# wandb_name = 'c3lsx34w'
# ckpt_name = 'best-epoch=488-step=26895.ckpt'

# model_variant = "control-sampling-flow-v5"
# wandb_name = 'hse612sq'
# ckpt_name = 'best-epoch=70-step=124000.ckpt'

# model_variant = "random-flow-v5"
# wandb_name = '6gs9tfto'
# ckpt_name = 'best-epoch=117-step=207000.ckpt'

# model_variant = "txpert-deloss"
# wandb_name = 'yveov9dj'
# ckpt_name = 'best-epoch=48-step=86436.ckpt'

model_variant = "random-flow-scale0.05-v4"
wandb_name = 'or98yqne'
ckpt_name = 'best-epoch=157-step=69678.ckpt'
scale = 0.05

# with open(f"configs/{config}", "r") as f:
#     cfg_dict = yaml.safe_load(f)
run = wandb.Api().run(f"park-hwa0826/TxPertzz/runs/{wandb_name}")
cfg_dict = run.config
cfg: DictConfig = OmegaConf.create(cfg_dict)

if 'noise_scale' not in cfg.model.flow_model:
    cfg.model.flow_model.noise_scale = scale

from rich.syntax import Syntax
from rich.console import Console
yaml_str = OmegaConf.to_yaml(cfg, resolve=True)
syntax = Syntax(yaml_str, "yaml", theme="github-light", line_numbers=True)
Console().print(syntax)


checkpoint_path = f"AAA_checkpoints/{cfg.datamodule.task_type}/{model_variant}_{wandb_name}/{ckpt_name}"

os.path.exists(checkpoint_path)

save_path = f"AAA_results/{cfg.datamodule.task_type}/{model_variant}/{wandb_name}_{ckpt_name.split('.')[0]}_predictions.pkl"
os.makedirs(os.path.dirname(save_path), exist_ok=True)


    1 mode: baseline                                                                                               
    2 seed: 1                                                                                                      
    3 debug: false                                                                                                 
    4 graph:                                                                                                       
    5   graph_cfg:                                                                                                 
    6     graph1:                                                                                                  
    7       mode: top_20                                                                                           
    8       graph_type: string                                                                                     
    9       norm_weights: false                                                                                    
   10       p_downsample: 1                                                                                        
   11       p_rewire_src: 0                                                                                        
   12       p_rewire_tgt: 0                                                                                        
   13       reduce2perts: true                                                                                     
   14 model:                                                                                                       
   15   lr: 0.0001                                                                                                 
   16   dropout: 0.2                                                                                               
   17   bce_loss: false                                                                                            
   18   omit_cntr: false                                                                                           
   19   cntr_model:                                                                                                
   20     rank: 16                                                                                                 
   21     model_type: mlp                                                                                          
   22     decode_latent: false                                                                                     
   23     use_batch_norm: true                                                                                     
   24     use_layer_norm: false                                                                                    
   25   flow_model:                                                                                                
   26     cond: control_mean                                                                                       
   27     model_type: random_noise                                                                                 
   28     latent_flow: false                                                                                       
   29     network_type: v4                                                                                         
   30     ae_checkpoint: null                                                                                      
   31     num_flow_steps: 50                                                                                       
   32     noise_scale: 0.05                                                                                        
   33   hidden_dim: 512                                                                                            
   34   latent_dim: 512                                                                                            
   35   model_type: txpertflow                          

In [6]:
from gspp.data.graphmodule import GSPGraph

from gspp.predictor import PertPredictor
from gspp.FlowPertPredictor import FlowPertPredictor

data_args = cfg.datamodule
datamodule = PertDataModule(**data_args)
datamodule.prepare_data(seed=cfg.seed)

graph = GSPGraph(
    pert2id=datamodule.pert2id,
    gene2id=datamodule.gene2id,
    **cfg.graph,
)
if 'flow' not in model_variant:
    model = PertPredictor.load_from_checkpoint(
        checkpoint_path,
        input_dim=datamodule.input_dim,
        output_dim=datamodule.output_dim,
        p_emb_dim=datamodule.p_emb_dim,
        adata_output_dim=datamodule.adata_output_dim,
        model_args=cfg.model,
        graph=graph,
    )
else:
    model = FlowPertPredictor.load_from_checkpoint(
        checkpoint_path,
        input_dim=datamodule.input_dim,
        output_dim=datamodule.output_dim,
        p_emb_dim=datamodule.p_emb_dim,
        adata_output_dim=datamodule.adata_output_dim,
        model_args=cfg.model,
        graph=graph,
        strict=False
    )

No perturbation embedding key provided
2025-12-22 01:37:42.627 | INFO     | gspp.data.datamodule:prepare_data:182 - Attempt loading data from /home/yinhua/CellPertQA/TxPert_zz/cache/K562_single_cell_line/de_adata_test.h5ad
2025-12-22 01:37:42.628 | INFO     | gspp.data.datamodule:prepare_data:185 - adata cached, loading from /home/yinhua/CellPertQA/TxPert_zz/cache/K562_single_cell_line/de_adata_test.h5ad
2025-12-22 01:37:47.956 | INFO     | gspp.data.datamodule:prepare_data:187 - Loading DEGs to uns
2025-12-22 01:37:48.680 | INFO     | gspp.data.datamodule:_process_treatment_data:289 - Filtered out the following perturbations: ['C7orf26+ctrl', 'C14orf178+ctrl', 'RPS10-NUDT3+ctrl', 'SEM1+ctrl', 'FAU+ctrl']
Categories (1092, object): ['AAMP+ctrl', 'AARS+ctrl', 'AATF+ctrl', 'ABCB7+ctrl', ..., 'ZNF335+ctrl', 'ZNF574+ctrl', 'ZNHIT6+ctrl', 'ZNRD1+ctrl']
2025-12-22 01:37:48.745 | INFO     | gspp.data.datamodule:_set_input_output_dimensions:315 - adata_output_dim :  5000
2025-12-22 01:37:51.49

In [7]:
from gspp.evaluation import evaluate
cfg.datamodule.batch_size = 2048
datamodule.setup('test')
results = evaluate(datamodule.test_dataloader(),model,model.device,datamodule.adata, datamodule.id2pert)

2025-12-22 01:38:28.246 | INFO     | gspp.data.datamodule:setup:346 - Setting up in stage test
5
Staring get the predictions...


100%|██████████| 19/19 [00:28<00:00,  1.50s/it]


In [8]:
import pickle

with open(save_path, "wb") as f:
    pickle.dump(results, f)
    print(f"saved to {save_path}")

saved to AAA_results/K562_single_cell_line/random-flow-scale0.05-v4/or98yqne_best-epoch=157-step=69678_predictions.pkl
